In [ ]:
import urbanpy as up
import geopandas as gpd
import matplotlib.pyplot as plt
import contextily as ctx
import rioxarray
import os
from tqdm.auto import tqdm
from shapely.geometry import Polygon, MultiPolygon

In [ ]:
amazonas = gpd.read_file("inputs/Amazonas")

In [ ]:
amazonas.crs.to_string()

In [ ]:
amazonas = amazonas.to_crs("EPSG:4326")

In [ ]:
amazonas.crs.to_string()

In [ ]:
ax = amazonas.plot(facecolor="none", edgecolor="black", linewidth=0.7)
ctx.add_basemap(ax, crs=amazonas.crs.to_string())
ax.set_title("Amazonas Limits")
ax.set_axis_off()

In [ ]:
amazonas["geometry"].explode()[0].area.sort_values(ascending=False)

In [ ]:
amazonas_gdf = gpd.GeoDataFrame(
    geometry=[amazonas["geometry"].explode()[0][122]], crs=amazonas.crs
)

In [ ]:
ax = amazonas_gdf.plot(facecolor="none", edgecolor="black", linewidth=0.7)
ctx.add_basemap(ax, crs=amazonas_gdf.crs.to_string())
ax.set_title("Amazonas Limits")
ax.set_axis_off()

In [ ]:
def convert_3D_2D(geometry):
    """
    Takes a GeoSeries of 3D Multi/Polygons (has_z) and returns a list of 2D Multi/Polygons
    """
    new_geo = []
    for p in geometry:
        if p.has_z:
            if p.geom_type == "Polygon":
                lines = [xy[:2] for xy in list(p.exterior.coords)]
                new_p = Polygon(lines)
                new_geo.append(new_p)
            elif p.geom_type == "MultiPolygon":
                new_multi_p = []
                for ap in p:
                    lines = [xy[:2] for xy in list(ap.exterior.coords)]
                    new_p = Polygon(lines)
                    new_multi_p.append(new_p)
                new_geo.append(MultiPolygon(new_multi_p))
    return new_geo

In [ ]:
amazonas_gdf_2d = convert_3D_2D(amazonas_gdf.geometry)

In [ ]:
amazonas_clean = gpd.GeoDataFrame(geometry=amazonas_gdf_2d, crs=amazonas.crs)

In [ ]:
amazonas_clean.crs

In [ ]:
amazonas_clean.to_parquet("outputs/amazonas_clean.parquet")

In [ ]:
# amazonas_hexs_1 = up.geom.gen_hexagons(resolution=1, city=amazonas_clean)
# amazonas_hexs_1.drop("geometry").to_csv("outputs/amazonas_hexs_1.csv", index=False)\

amazonas_hexs_2 = up.geom.gen_hexagons(resolution=2, city=amazonas_clean)
amazonas_hexs_2.drop("geometry", axis=1).to_csv(
    "outputs/amazonas_hexs_2.csv", index=False
)

In [ ]:
amazonas_hexs_3 = up.geom.gen_hexagons(resolution=3, city=amazonas_clean)
amazonas_hexs_3.drop("geometry", axis=1).to_csv(
    "outputs/amazonas_hexs_3.csv", index=False
)

amazonas_hexs_4 = up.geom.gen_hexagons(resolution=4, city=amazonas_clean)
amazonas_hexs_4.drop("geometry", axis=1).to_csv(
    "outputs/amazonas_hexs_4.csv", index=False
)

In [ ]:
fig, (ax, ax1) = plt.subplots(1, 2, figsize=(20, 10))

amazonas_gdf.plot(facecolor="none", edgecolor="black", linewidth=0.7, ax=ax)
amazonas_hexs_4.plot(ax=ax, facecolor="none", edgecolor="red", linewidth=0.3)
ctx.add_basemap(ax, crs=amazonas_gdf.crs.to_string())
ax.set_axis_off()

amazonas_gdf.plot(facecolor="none", edgecolor="black", linewidth=0.7, ax=ax1)
amazonas_hexs_4.plot(ax=ax1, facecolor="none", edgecolor="red", linewidth=0.3)
ctx.add_basemap(ax1, crs=amazonas_gdf.crs.to_string())
ax1.set_axis_off()

plt.tight_layout()
plt.savefig(
    "outputs/amazonas_hexs_4_2maps.png", dpi=300, bbox_inches="tight", format="png"
)

In [ ]:
ax = amazonas_gdf.plot(
    facecolor="none", edgecolor="black", linewidth=0.7, figsize=(10, 10)
)
amazonas_hexs_4.plot(ax=ax, facecolor="none", edgecolor="red", linewidth=0.3)
ctx.add_basemap(ax, crs=amazonas_gdf.crs.to_string())
ax.set_title("Amazonas Hexs Res 4")
ax.set_axis_off()

plt.savefig("outputs/amazonas_hexs_4.svg", dpi=300, bbox_inches="tight", format="svg")

In [ ]:
amazonas_hexs_5 = up.geom.gen_hexagons(resolution=5, city=amazonas_clean)
amazonas_hexs_5.shape

In [ ]:
amazonas_hexs_5.

In [ ]:
amazonas_hexs_5.to_parquet("outputs/amazonas_hexs_5.parquet")

In [ ]:
amazonas_hexs_6 = up.geom.gen_hexagons(resolution=6, city=amazonas_clean)
amazonas_hexs_6.shape

In [ ]:
amazonas_hexs_6.to_parquet("outputs/amazonas_hexs_6.parquet")

In [ ]:
amazonas_hexs_7 = up.geom.gen_hexagons(resolution=7, city=amazonas_clean)
amazonas_hexs_7.shape

In [ ]:
amazonas_hexs_7.to_parquet("outputs/amazonas_hexs_7.parquet")

In [ ]:
# Takes too much time
# amazonas_hexs_8 = up.geom.gen_hexagons(resolution=5, city=amazonas_clean)
# amazonas_hexs_8.shape

In [ ]:
country_boundaries = gpd.read_file("inputs/Cartographic Boundary Files/LAC/level 0")

In [ ]:
country_boundaries.plot()

In [ ]:
country_boundaries.head()

In [ ]:
country_boundaries["ADM0_PCODE"].unique()

In [ ]:
amzn_countries = country_boundaries[
    country_boundaries["ADM0_PCODE"].isin(
        ["BO", "BR", "CO", "EC", "GY", "PE", "SR", "VE"]
    )
]

In [ ]:
ax = amazonas_gdf.plot(facecolor="none", edgecolor="black", linewidth=0.7)
amzn_countries.plot("ADM0_PCODE", ax=ax, alpha=0.5)
ctx.add_basemap(ax, crs=amazonas_gdf.crs.to_string())
ax.set_title("Amazon Countries")
ax.set_axis_off()

## WoldPop - Age and sex structures

- Resolution: 100m^2
- Year: 2020
- Classes: 5-year age groups + <1 year
- Version: Constrained


# Raster based version of population data download


In [ ]:
worldpop_data = "inputs/WorldPop"
countries = ["PER", "COL", "GUY", "SUR", "VEN", "BOL", "ECU"]  # "BRA"
age_groups = [5, 10, 15]
genders = ["m", "f"]

In [ ]:
def merge_rio_hex(
    hexs: gpd.GeoDataFrame,
    clip_geometries: gpd.GeoDataFrame,
    rio_filename: str,
    data_name: str,
    agg: str,
    band: int = 1,
) -> gpd.GeoDataFrame:
    """
    Merge raster data with hexagons.

    Parameters
    ----------
    hexs : gpd.GeoDataFrame
        GeoDataFrame with hexagons.
    rio_filename : str
        Filename of the raster data.
    agg : str
        Aggregation method to use.

    Returns
    -------
    gpd.GeoDataFrame
        GeoDataFrame with the merged data.
    """
    rds = rioxarray.open_rasterio(rio_filename)
    rds.rio.set_crs(hexs.crs.to_string())
    clipped = rds.rio.clip(clip_geometries, rds.rio.crs)
    rio_data = clipped.sel(band=band).drop_vars("band")
    rio_data.name = data_name
    rio_data_df = rio_data.to_pandas().unstack().reset_index()
    rio_data_df.columns = ["x", "y", data_name]
    rio_data_gdf = gpd.GeoDataFrame(
        rio_data_df,
        geometry=gpd.points_from_xy(rio_data_df.x, rio_data_df.y),
        crs=rio_data.rio.crs.to_string(),
    )
    rio_data_gdf = rio_data_gdf.dropna(
        subset=[data_name]
    )  # Drop NaN values for faster processing
    agg_dict = {data_name: agg}
    return up.geom.merge_shape_hex(hexs=hexs, shape=rio_data_gdf, agg=agg_dict)

## Ecuador


In [ ]:
ecuador_bounds = amzn_countries[amzn_countries["ADM0_PCODE"] == "EC"]
ecuador_bounds

In [ ]:
temp = ecuador_bounds.explode().loc[10]
temp["area"] = temp.area
temp.plot("area")

In [ ]:
# Filter continental ecuador
ecuador_gdf = temp.cx[-82.5:-75, -5:-0.5]

In [ ]:
ecuador_hexs = up.geom.gen_hexagons(resolution=7, city=ecuador_gdf)

In [ ]:
print("Number of hexagons:", ecuador_hexs.shape[0])

In [ ]:
ax = ecuador_hexs.plot(facecolor="none", edgecolor="black", linewidth=0.25)
ctx.add_basemap(ax, crs=ecuador_hexs.crs.to_string())
ax.set_axis_off()
plt.show()

In [ ]:
progress_bar = tqdm(total=len(genders) * len(age_groups))
country_string = "ecu"
year = "2020"
for gender in genders:
    for age in age_groups:
        filename = f"{worldpop_data}/{country_string}/{country_string}_{gender}_{age}_{year}.tif"
        assert os.path.exists(filename), f"File {filename} does not exist"
        ecuador_hexs = merge_rio_hex(
            hexs=ecuador_hexs,
            clip_geometries=ecuador_gdf.geometry,
            rio_filename=filename,
            data_name=f"{country_string}_{gender}_{age}_{year}",
            agg="sum",
            band=1,
        )
        progress_bar.update(1)
progress_bar.close()